In [1]:
import importlib
import os
from itertools import product

from dotenv import load_dotenv
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.tuner import Tuner

load_dotenv("../env")
seed_everything(42)

Seed set to 42


42

In [2]:
import luminar.mongo

importlib.reload(luminar.mongo)
from luminar.mongo import MongoDBAdapter

In [3]:
# domain = "student_essays"
# db = MongoDBAdapter(
#     os.environ.get("MONGO_DB_CONNECTION"),
#     "prismai",
#     "collected_items",
#     "synthesized_texts",
#     "transition_scores",
#     domain=domain,
#     source_collection_limit=1500,
# )

In [4]:
import luminar.document.data
import luminar.features

importlib.reload(luminar.document.data)
importlib.reload(luminar.features)

from luminar.document.data import DocumentClassificationDataModule
from luminar.features import OneDimFeatures, ThreeDimFeatures, TwoDimFeatures

In [5]:
import luminar.document.model

importlib.reload(luminar.document.model)

from luminar.document.model import ConvolutionalLayerSpec, DocumentClassficationModel

In [6]:
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

In [ ]:
# for bs, fs in product((32, 64, 128), (128, 256, 512)):
for feature_size in (64, 128, 256):
    feature_size = OneDimFeatures(feature_size)
    for domain in (
        # "arxiv_papers",
        "blog_authorship_corpus",
        "bundestag",
        "cnn_news",
        "euro_court_cases",
        # "gutenberg",
        "house_of_commons",
        # "spiegel_articles",
        "student_essays",
    ):
        db = MongoDBAdapter(
            os.environ.get("MONGO_DB_CONNECTION"),
            "prismai",
            "collected_items",
            "synthesized_texts",
            "transition_scores",
            domain=domain,
            source_collection_limit=1500,
        )
        seed_everything(42)
        dm = DocumentClassificationDataModule(db, feature_size)
        for lr, pdim in product((0.0005, 0.0001), (32, 64, 128)):
            # for lr, wu, pr in product((1e-3, 1e-4, 1e-5), (0, 50, 100), (64, 128, 256)):
            seed_everything(42)
            model = DocumentClassficationModel(
                dm.feature_selection.effective_shape(),
                projection_dim=128,
                learning_rate=lr,
                warmup_steps=100,
            )
            trainer = Trainer(
                max_epochs=50,
                logger=pl_loggers.TensorBoardLogger(
                    save_dir="logs/",
                    name=domain,
                ),
                gradient_clip_val=0.5,
                deterministic=True,
                callbacks=[EarlyStopping(monitor="val_loss", mode="min")],
            )
            # tuner = Tuner(trainer)
            trainer.fit(model, dm)
            (metrics,) = trainer.validate(model, dataloaders=[dm.val_dataloader()])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Caching Enabled - Dataset Already Cached
Caching Enabled - Loading Dataset in setup(TrainerFn.FITTING)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.

  | Name        | Type              | Params | Mode  | In sizes   | Out sizes
-----------------------------------------------------------------------------------
0 | conv_layers | ModuleList        | 148 K  | train | ?          | ?        
1 | classifier  | Sequential        | 2.1 M  | train | [1, 16384] | [1, 1]   
2 | criterion   | BCEWithLogitsLoss | 0      | train | ?          | ?        
-----------------------------------------------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1188: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Caching Enabled - Dataset Already Cached


/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│            acc            │     0.910909116268158     │
│          roc_auc          │     0.967555820941925     │
│         val_loss          │    0.20930875837802887    │
└───────────────────────────┴───────────────────────────┘

In [12]:
# tuner.lr_find(model, datamodule=dm)

In [13]:
trainer.fit(model, dm)
metrics, = trainer.validate(model, dataloaders=[dm.val_dataloader()])

/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory logs/student_essays/version_3/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.

  | Name        | Type              | Params | Mode  | In sizes   | Out sizes
-----------------------------------------------------------------------------------
0 | conv_layers | ModuleList        | 148 K  | train | ?          | ?        
1 | classifier  | Sequential        | 2.1 M  | train | [1, 16384] | [1, 1]   
2 | criterion   | B

Caching Enabled - Dataset Already Cached


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1188: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Caching Enabled - Dataset Already Cached


/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│            acc            │    0.9018181562423706     │
│          roc_auc          │    0.9587038159370422     │
│         val_loss          │     0.242477685213089     │
└───────────────────────────┴───────────────────────────┘